The following notebook shows how we can use a VLLM server that follows OpenAI RESTful API. It considers:

* **Using directly the OpenAI library.** It has the advantage that we can add extra parameters supported specifically by VLMM but not by the "traditional" OpenAI API. They are introduced in the `extra_body` part of the request.

* **Using LangChain with `ChatOpenAI`**. In this case, we need to follow the traditional approach, we have not been able to use the extra features of  VLLM

**Note on LoRA adapters:** I think ChatOpenAI should work with LoRA adapters because they are considered "another model" of the server. We probably need one docker image per model, with its set of associated LoRA adapters.

## Using OpenAI to directly interact with the VLLM server:

In [25]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="NousResearch/Meta-Llama-3-8B-Instruct",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", role='assistant', function_call=None, tool_calls=None)


#### Extra parameters for Chat API

[vLLM supports a set of parameters that are not part of the OpenAI API. In order to use them, you can pass them as extra parameters in the OpenAI client. Or directly merge them into the JSON payload if you are using HTTP call directly.](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#extra-parameters)

#### Without `guided_choice`

In [26]:
completion = client.chat.completions.create(
  model="NousResearch/Meta-Llama-3-8B-Instruct",
  messages=[
    {"role": "user", "content": "Classify this sentiment: vLLM is wonderful!"}
  ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The sentiment in the statement "vLLM is wonderful!" is POSITIVE.', role='assistant', function_call=None, tool_calls=None)


#### With `guided_choice`

In [27]:
completion = client.chat.completions.create(
  model="NousResearch/Meta-Llama-3-8B-Instruct",
  messages=[
    {"role": "user", "content": "Classify this sentiment: vLLM is wonderful!"}
  ],
  extra_body={
    "guided_choice": ["positive", "negative"]
  }
)

print(completion.choices[0].message)

ChatCompletionMessage(content='positive', role='assistant', function_call=None, tool_calls=None)


## Using LangChain to interact with the VLLM server

In [11]:
from langchain_openai import ChatOpenAI


inference_server_url = "http://localhost:8000/v1"

chat = ChatOpenAI(
    model="NousResearch/Meta-Llama-3-8B-Instruct",
    openai_api_key="EMPTY",
    openai_api_base=inference_server_url,
    max_tokens=100,
    temperature=0,
)

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to Italian."
    ),
    HumanMessage(
        content="Translate the following sentence from English to Italian: I love programming."
    ),
]
chat.invoke(messages)

AIMessage(content='The translation of the sentence "I love programming" from English to Italian is:\n\n"Mi piace programmazione."\n\nHere\'s a breakdown of the translation:\n\n* "I" is translated to "Mi"\n* "love" is translated to "piace"\n* "programming" is translated to "programmazione"', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 40, 'total_tokens': 105}, 'model_name': 'NousResearch/Meta-Llama-3-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eed052d5-9214-4fdb-b96b-0ade934b60bd-0')

In [13]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

template = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English", output_language="Italian", text="I love programming."
    ).to_messages()
)

AIMessage(content='Ti piace il programming!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 31, 'total_tokens': 38}, 'model_name': 'NousResearch/Meta-Llama-3-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-801bdad5-8d97-4462-8734-81ec6cc20c39-0')

### Extra Parameters

**I have not been able to call the server with extra parameters when using LangChain**. If we want all of the extra features that VLLM offers, we would probably need to extend ChatOpenAI

#### Without `guided_choice`

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(
        content="You are a helpful assistant. Classify this sentiment: vLLM is wonderful!"
    ),
]
chat.invoke(messages)

AIMessage(content='I would classify this sentiment as POSITIVE. The use of the word "wonderful" is a strong positive adjective that expresses enthusiasm and admiration for the topic being referred to, which is vLLM.', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 33, 'total_tokens': 76}, 'model_name': 'NousResearch/Meta-Llama-3-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1f5f9ec9-646f-4f32-8f52-45f2645e07ef-0')

#### With `guided_choice`

Try #1 (kwargs)

In [35]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(
        content="You are a helpful assistant. Classify this sentiment: vLLM is wonderful!"
    ),
]

extra_body ={
    "guided_choice": ["positive", "negative"]
  }

chat.invoke(input=messages, kwargs=extra_body)

SyntaxError: invalid syntax (1532967881.py, line 13)

#### With `guided_choice`

In [23]:
chat.with_config(
    configurable={"guided_choice": ["positive", "negative"]}
).invoke(messages)

AIMessage(content='I would classify this sentiment as POSITIVE. The use of the word "wonderful" is a strong positive adjective that expresses enthusiasm and admiration for the topic being referred to, which is vLLM.', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 33, 'total_tokens': 76}, 'model_name': 'NousResearch/Meta-Llama-3-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d3c8399b-ba31-4481-a326-009dc8c1da76-0')

In [30]:
# Assuming you have LangChain configured with your API key
client = ChatOpenAI(
    model="NousResearch/Meta-Llama-3-8B-Instruct",
    openai_api_key="EMPTY",
    openai_api_base=inference_server_url,
    max_tokens=100,
    temperature=0,
)

# Define the conversation with prompts and guided choices
conversation = [
    {
        "role": "user",
        "content": "Classify this sentiment: vLLM is wonderful!",
        "arguments": {"guided_choice": ["positive", "negative"]},
    }
]

# Use ChatOpenAI's chat functionality with the conversation object
completion = client(conversation)

# Print the first response from the model
print(completion[0]["message"])

AttributeError: 'dict' object has no attribute 'content'